In [1]:
import csv
from operator import itemgetter
import networkx as nx
from networkx.algorithms import community

# Read data

In [2]:
with open('www.forefdn.org - Nodes.csv', 'r') as nodecsv: # Open the file
    nodereader = csv.reader(nodecsv) # Read the csv
    # Retrieve the data (using Python list comprhension and list slicing to remove the header row, see footnote 3)
    nodes = [n for n in nodereader][1:]

node_names = [n[0] for n in nodes] # Get a list of only the node names

In [3]:
with open('www.forefdn.org - Edges (External).csv', 'r') as edgecsv: # Open the file
    edgereader = csv.reader(edgecsv) # Read the csv
    edges = [tuple(e) for e in edgereader][1:] # Retrieve the data

# Build Network

In [4]:
G = nx.Graph()
G.add_nodes_from(node_names)
G.add_edges_from(edges)

In [5]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 766
Number of edges: 1433
Average degree:   3.7415


In [6]:
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree')

# Metrics

In [7]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.004890868444854007


In [8]:
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

Triadic closure: 0


In [9]:
betweenness_dict = nx.betweenness_centrality(G) 
eigenvector_dict = nx.eigenvector_centrality(G)

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [ ]:
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

In [10]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)

Top 20 nodes by betweenness centrality:
('https://twitter.com/FOREfdn', 0.10226017618088781)
('https://www.linkedin.com/company/forefoundation/', 0.10226017618088781)
('https://www.youtube.com/channel/UCD1U3OmYsaNDmv5nSiRbkAA', 0.10226017618088781)
('http://eepurl.com/gbKcc1', 0.10226017618088781)
('https://forefdn.org/our-grantees/', 0.027833997468557557)
('https://forefdn.org/our-grantees/page/4/', 0.027833997468557557)
('https://forefdn.org/our-grantees/page/5/', 0.027833997468557557)
('https://forefdn.org/our-grantees/page/2/', 0.027833997468557557)
('https://forefdn.org/our-grantees/page/3/', 0.027833997468557557)
('https://forefdn.org/tapping-into-virtual-recovery-supports/', 0.025086728672174315)
('https://forefdn.org/grantee-spotlight-improving-access-to-recovery-supports-in-colleges-and-universities/', 0.02099639615132926)
('https://forefdn.org/grantee-spotlight-encouraging-physicians-to-treat-opioid-use-disorder-in-emergency-departments-across-the-u-s/', 0.016685993159345193)

In [11]:
#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

Name: https://twitter.com/FOREfdn | Betweenness Centrality: 0.10226017618088781 | Degree: 135
Name: https://www.linkedin.com/company/forefoundation/ | Betweenness Centrality: 0.10226017618088781 | Degree: 135
Name: https://www.youtube.com/channel/UCD1U3OmYsaNDmv5nSiRbkAA | Betweenness Centrality: 0.10226017618088781 | Degree: 135
Name: http://eepurl.com/gbKcc1 | Betweenness Centrality: 0.10226017618088781 | Degree: 135
Name: https://forefdn.org/our-grantees/ | Betweenness Centrality: 0.027833997468557557 | Degree: 98
Name: https://forefdn.org/our-grantees/page/4/ | Betweenness Centrality: 0.027833997468557557 | Degree: 98
Name: https://forefdn.org/our-grantees/page/5/ | Betweenness Centrality: 0.027833997468557557 | Degree: 98
Name: https://forefdn.org/our-grantees/page/2/ | Betweenness Centrality: 0.027833997468557557 | Degree: 98
Name: https://forefdn.org/our-grantees/page/3/ | Betweenness Centrality: 0.027833997468557557 | Degree: 98
Name: https://forefdn.org/tapping-into-virtual-re

# Community Detection

In [12]:
communities = community.greedy_modularity_communities(G)

In [13]:
modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'modularity')

In [14]:
# First get a list of just the nodes in that class
class0 = [n for n in G.nodes() if G.nodes[n]['modularity'] == 0]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G.nodes[n]['eigenvector'] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector[:5]:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])

KeyError: 'eigenvector'

In [15]:
for i,c in enumerate(communities): # Loop through the list of communities
    if len(c) > 2: # Filter out modularity classes with 2 or fewer nodes
        print('Class '+str(i)+':', list(c)) # Print out the classes and their members

Class 0: ['https://forefdn.orgcontact-us/', 'https://forefdn.org/news-updates/page/2/?res=all', 'https://forefdn.org/resources/', 'https://forefdn.org/resources/?res=issue-brief', 'https://youtu.be/Xqu5mW_1AdE', 'https://forefdn.org/careers/', 'https://forefdn.org/commentary-best-practices-for-delivering-moud-in-correctional-settings-during-the-pandemic/', 'https://www.facebook.com/sharer/sharer.php?u=https://forefdn.org/webinar-oud-and-the-emergency-department-experience-during-the-covid-19-pandemic-4-30-2020', 'https://www.hhs.gov/hipaa/for-professionals/special-topics/emergency-preparedness/notification-enforcement-discretion-telehealth/index.html', 'https://ssrs.com/', 'https://forefdn.org/', 'https://forefdn.org/about-us/', 'https://www.samhsa.gov/sites/default/files/otp-guidance-20200316.pdf', 'https://www.linkedin.com/shareArticle?mini=true&url=https://forefdn.org/webinar-the-role-of-peer-recovery-coaches-and-navigators-during-the-covid-19-pandemic-5-28-2020&title=Webinar:%20The

In [16]:
nx.write_gexf(G, 'forefdn_external.gexf')